In [161]:
# 기초 전처리
import pandas as pd
import numpy as np
import os

# 크롤링
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

# 컬럼 전체 확인 가능하도록 출력 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

# pandas 결과값의 표현 범위 소수점 2자리수로 변경
pd.options.display.float_format = '{:.2f}'.format

# 파일 로드위한 directory 확인 및 현재 경로로 설정
a = os.getcwd()
os.chdir(a)

In [162]:
store = pd.read_csv("소상공인시장진흥공단_상가(상권)정보_서울_202106.csv", encoding='utf-8')
store_by_admin = pd.read_csv('행정동기준_상권코드.csv', encoding = 'cp949', index_col = 0)

# 데이터 크롤링

In [163]:
# 크롤링용 df 따로 준비
df = store.copy()

< 리뷰 크롤링 테스트를 위해 샘플 데이터 뽑기>
- 아래의 중분류중 하나를 골라서 해당 데이터로만 리뷰 뽑기 실행

음식배달서비스     79


실외운동시설      74


캠프/별장/펜션    49



종교용품판매      48


놀이/여가/취미    38



스포츠/운동      30


유스호스텔        9


부동산관련서비스     3

In [164]:
# 실험 위해 놀이 / 여가/ 취미 카테고리 데이터만 수집
df = df.loc[df['상권업종중분류명'] == '놀이/여가/취미'] 
df = df[['상호명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명', '행정동명', '위도', '경도']]

In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 5332 to 306345
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   상호명       38 non-null     object 
 1   상권업종중분류명  38 non-null     object 
 2   상권업종소분류명  38 non-null     object 
 3   표준산업분류명   38 non-null     object 
 4   행정동명      38 non-null     object 
 5   위도        38 non-null     float64
 6   경도        38 non-null     float64
dtypes: float64(2), object(5)
memory usage: 2.4+ KB


In [166]:
# 컬럼명 단순화
df.columns = ['name',  # 상호명
              'cate_1',  # 중분류명
              'cate_2',  # 소분류명
              'cate_3',  # 표준산업분류명
              'dong',  # 행정동명
              'lon',  # 위도
              'lat'  # 경도
              ]

## 매장별 상세페이지 url수집

In [167]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time


#chromedriver = '/Users/datakim/workspace/selenium_learning/chromedriver' 
#driver = webdriver.Chrome(chromedriver) 
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install())  # chromedriver 열기


df['kakao_keyword'] = df['dong'] + "%20" + df['name']
df['kakao_map_url'] = ''


for i, keyword in enumerate(df['kakao_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0]} 행", keyword)
    try:
        kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"
        driver.get(kakao_map_search_url)
        time.sleep(0.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#info\.search\.place\.list > li:nth-child(1) > div.info_item > div.contact.clickArea > a.moreview").get_attribute('href')
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # child(1)이 없던데요?
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.moreview").get_attribute('href')
                time.sleep(1)
            except Exception as e2:
                print('뽑을 url이 없습니다! error 내용 :',e2)
                df.iloc[i,-1] = np.nan
                time.sleep(0.5)
        else:
            pass



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\HB\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


이번에 찾을 키워드 : 0 / 38 행 대치4동%20하나투어
이번에 찾을 키워드 : 1 / 38 행 종암동%20성원노래연습실
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.moreview"}
  (Session info: chrome=94.0.4606.61)

이번에 찾을 키워드 : 2 / 38 행 잠실본동%20못된강아지
이번에 찾을 키워드 : 3 / 38 행 대치4동%20리틀치프코리아
이번에 찾을 키워드 : 4 / 38 행 방학3동%20정의공주텃밭농장
이번에 찾을 키워드 : 5 / 38 행 역삼1동%20지스코산업연구소
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.moreview"}
  (Session info: chrome=94.0.4606.61)

이번에 찾을 키워드 : 6 / 38 행 청구동%20금선
이번에 찾을 키워드 : 7 / 38 행 구로5동%20알파고보드카페
이번에 찾을 키워드 : 8 / 38 행 방학3동%20레포츠투어
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.moreview"}
  (Session info: chrome=94.0.4606.61)

이번에 찾을 키워드 : 9 / 38 행 개봉1동%20대따커낚시

In [168]:
# 상세페이지 없는 곳 = 폐업 가능성 높음 = 이상치로 처리
df.dropna(inplace=True)
df.head()

,name,cate_1,cate_2,cate_3,dong,lon,lat,kakao_keyword,kakao_map_url
5332,하나투어,놀이/여가/취미,기타오락관련산업,그외 기타 분류안된 오락관련 서비스업,대치4동,37.50,127.05,대치4동%20하나투어,https://place.map.kakao.com/26602645
14515,못된강아지,놀이/여가/취미,기타오락관련산업,그외 기타 분류안된 오락관련 서비스업,잠실본동,37.51,127.09,잠실본동%20못된강아지,https://place.map.kakao.com/1689809520
14644,리틀치프코리아,놀이/여가/취미,기타오락관련산업,그외 기타 분류안된 오락관련 서비스업,대치4동,37.50,127.05,대치4동%20리틀치프코리아,https://place.map.kakao.com/72205729
16313,정의공주텃밭농장,놀이/여가/취미,주말농장,기타 부동산 임대업,방학3동,37.66,127.02,방학3동%20정의공주텃밭농장,https://place.map.kakao.com/27006326
23400,금선,놀이/여가/취미,기타오락관련산업,그외 기타 분류안된 오락관련 서비스업,청구동,37.56,127.02,청구동%20금선,https://place.map.kakao.com/8205407


In [169]:
df2 = df.copy()

## 상세 페이지에서 리뷰 수집

In [172]:
# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합칠 것입니다.
from tqdm import tqdm_notebook
from webdriver_manager.chrome import ChromeDriverManager
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install())  # chromedriver 열기

# 각 데이터를 담을 곳
columns = ['store','comment','rating','review_date']
result = pd.DataFrame(columns=columns)

# df의 url을 돌며 리뷰 수집시작
for i, url in enumerate(tqdm_notebook(df['kakao_map_url'])):
    driver.get(url) # df에 있는 url 가져오기
    time.sleep(1)
    
    # 첫 페이지 리뷰 목록 찾기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_lists = soup.select('.list_evaluation > li') 
    
    if len(review_lists) != 0: # 리뷰가 있다면 수집
        for idx, review in enumerate(review_lists):
            # 리뷰 내용 가져오기 (상호명, 리뷰, 별점, 리뷰 작성일)
            store = driver.find_element_by_css_selector("#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > h2").text # 상호명
            comment = review.select('.txt_comment > span')[0].text
            rating = rating = review.select('.grade_star > em')[0].text
            review_date = driver.find_element_by_css_selector("#mArticle > div.cont_evaluation > div.evaluation_review > ul > li > div.comment_info > div > span.time_write").text
            #print(store)
            #print(comment)
            #print(rating)
            #print(review_date)
            result = result.append(pd.DataFrame([[store, comment, rating, review_date]],columns= ['store','comment','rating','review_date']), ignore_index=True)
            print("현재 매장의", idx+1, "번째 review를 수집했습니다.")
    else: # 리뷰없다면 표시
        print("이번 매장은 리뷰가 없네요ㅠㅠ")
        #result.set_index('store', inplace = True)

#df['kakao_store_name'] = kakao_map_name_list[0]  # 카카오 상세페이지에서 크롤링한 상호명
#df['kakao_star_point'] = kakao_map_star_review_stars_list[0]  # 카카오 상세페이지에서 평가한 별점 평점
#df['kakao_star_point_qty'] = kakao_map_star_review_qty_list  # 카카오 상세페이지에서 별점 평가를 카운트한 횟수
#df['kakao_map_star_review_stars_list'] = blog_review_list  # 카카오 상세페이지에 나온 블로그 리뷰 텍스트들
#df['kakao_map_star_review_qty_list'] = blog_review_qty_list  # 카카오 상세페이지에 나온 블로그 리뷰의 총 개수

driver.quit()
result



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\HB\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


  0%|          | 0/21 [00:00<?, ?it/s]

3 번째 매장은 리뷰가 없네요ㅠㅠ
현재 매장의 1 번째 review를 수집했습니다.
0 번째 매장은 리뷰가 없네요ㅠㅠ
0 번째 매장은 리뷰가 없네요ㅠㅠ
0 번째 매장은 리뷰가 없네요ㅠㅠ
현재 매장의 1 번째 review를 수집했습니다.
현재 매장의 2 번째 review를 수집했습니다.
현재 매장의 3 번째 review를 수집했습니다.
2 번째 매장은 리뷰가 없네요ㅠㅠ
2 번째 매장은 리뷰가 없네요ㅠㅠ
2 번째 매장은 리뷰가 없네요ㅠㅠ
현재 매장의 1 번째 review를 수집했습니다.
현재 매장의 2 번째 review를 수집했습니다.
현재 매장의 3 번째 review를 수집했습니다.
현재 매장의 4 번째 review를 수집했습니다.
3 번째 매장은 리뷰가 없네요ㅠㅠ
3 번째 매장은 리뷰가 없네요ㅠㅠ
3 번째 매장은 리뷰가 없네요ㅠㅠ
현재 매장의 1 번째 review를 수집했습니다.
현재 매장의 2 번째 review를 수집했습니다.
현재 매장의 3 번째 review를 수집했습니다.
현재 매장의 4 번째 review를 수집했습니다.
현재 매장의 5 번째 review를 수집했습니다.
4 번째 매장은 리뷰가 없네요ㅠㅠ
현재 매장의 1 번째 review를 수집했습니다.
현재 매장의 2 번째 review를 수집했습니다.
현재 매장의 3 번째 review를 수집했습니다.
현재 매장의 4 번째 review를 수집했습니다.
현재 매장의 5 번째 review를 수집했습니다.
4 번째 매장은 리뷰가 없네요ㅠㅠ
현재 매장의 1 번째 review를 수집했습니다.
현재 매장의 2 번째 review를 수집했습니다.
현재 매장의 3 번째 review를 수집했습니다.
현재 매장의 4 번째 review를 수집했습니다.
3 번째 매장은 리뷰가 없네요ㅠㅠ
3 번째 매장은 리뷰가 없네요ㅠㅠ
3 번째 매장은 리뷰가 없네요ㅠㅠ


,store,comment,rating,review_date
0,못된강아지 신천점,진짜 최악!!!! 무슨 지우개를 고무줄로해서 3개씩 묶어놓음? 양심이 있으면 ...,1점,2019.10.05.
1,알파고보드카페,0428방문 원형으로 되어있어서 특이했어요 겜블러된기분ㅋㅋㅋㅋ 마침 딱 아이유 코...,5점,2021.04.29.
2,알파고보드카페,입구에 들어갔다가 뛰쳐나왔다....악의 소굴인줄...건물에서 악취나고....업장에선...,1점,2021.04.29.
3,알파고보드카페,게임이 너무 적고 방 안에서 담배 냄새가 너무 심해요 음료 값이 너무 비쌈..,2점,2021.04.29.
4,코난방탈출 구로디지털단지역점,돈아까워요,1점,2021.06.13.
5,코난방탈출 구로디지털단지역점,"여기서 무난한 거 하나 했는데, 퀄리티☆ 개연성☆ 역대간 방탈출 중에 제일 규모도...",1점,2021.06.13.
6,코난방탈출 구로디지털단지역점,씹노잼 제일어려운거 했는데 너무 쉽고 깨고나서도 이 돈 내고 왜했나 현타옴 알바생도...,1점,2021.06.13.
7,코난방탈출 구로디지털단지역점,편안하고 좋아요,5점,2021.06.13.
8,메종드시크릿 방탈출카페,방탈출 마니아로서 좀 별로였어요,1점,2021.09.03.
9,메종드시크릿 방탈출카페,스토리 좋음 방탈출 덕후인데 여긴 인정 ㅋㅋ,5점,2021.09.03.
